## import Libraries

In [57]:
import os
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## Define paths and read date

In [76]:
# Get the absolute path of the current directory
base_dir = os.path.abspath('.')

# Join the base directory with the relative path of the directories
cardboard_dir = os.path.join(base_dir, 'TrashType_Image_Dataset', 'cardboard')
glass_dir = os.path.join(base_dir, 'TrashType_Image_Dataset', 'glass')
metal_dir = os.path.join(base_dir, 'TrashType_Image_Dataset', 'metal')
paper_dir = os.path.join(base_dir, 'TrashType_Image_Dataset', 'paper')
plastic_dir = os.path.join(base_dir, 'TrashType_Image_Dataset', 'plastic')
trash_dir = os.path.join(base_dir, 'TrashType_Image_Dataset', 'trash')

# Define the paths to the directories
paths = [cardboard_dir, glass_dir, metal_dir, paper_dir, plastic_dir, trash_dir]

# Define data augmentation parameters
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Function to load images from directory and apply data augmentation
def load_images_from_directory(directory, batch_size=32):
    generator = datagen.flow_from_directory(
        directory,
        batch_size=batch_size,
        class_mode='categorical'
    )
    return generator

# Load augmented images for training and validation
train_generators, validation_generators = [], []
for path in paths:
    train_generator = load_images_from_directory(path, batch_size=32)
    validation_generator = load_images_from_directory(path, batch_size=32)
    x, y = train_generator.next()
    validation_x, validation_y = validation_generator.next()
    train_generators.append((x, y))
    validation_generators.append((validation_x, validation_y))

# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit_generator(
    train_generators,
    steps_per_epoch=2000,
    epochs=50,
    validation_data=validation_generators,
    validation_steps=800
)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


C:\Users\shahr\AppData\Local\Temp\ipykernel_10940\2507964604.py:58: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


ValueError: Data is expected to be in format `x`, `(x,)`, `(x, y)`, or `(x, y, sample_weight)`, found: (<keras.src.preprocessing.image.DirectoryIterator object at 0x000002B6EFC43AD0>, <keras.src.preprocessing.image.DirectoryIterator object at 0x000002B6DB52D310>, <keras.src.preprocessing.image.DirectoryIterator object at 0x000002B68221B450>, <keras.src.preprocessing.image.DirectoryIterator object at 0x000002B6821F8190>, <keras.src.preprocessing.image.DirectoryIterator object at 0x000002B689245050>, <keras.src.preprocessing.image.DirectoryIterator object at 0x000002B689451B10>)